<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 10, 2025</h2>

<p style="font-size:16pt">
For each rosette, 100 random data samples are saved.

In [66]:
from astropy.table import Table, vstack
import pandas as pd
import numpy as np
from astropy.cosmology import Planck18
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import ascii

## Real data

In [34]:
data_n = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_clustering.dat.fits")
data_s = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_clustering.dat.fits")

names_n = [name for name in data_n.colnames if len(data_n[name].shape) <= 1]
names_s = [name for name in data_n.colnames if len(data_n[name].shape) <= 1]

data_n_filt = data_n[names_n].to_pandas()
data_s_filt = data_s[names_s].to_pandas()

In [37]:
sizes_n = {}
for i in set(data_n_filt['ROSETTE_NUMBER']):
    df_rosette = data_n_filt[data_n_filt['ROSETTE_NUMBER']==i]
    sizes_n[i] = len(df_rosette)

sizes_s = {}
for i in set(data_s_filt['ROSETTE_NUMBER']):
    df_rosette = data_s_filt[data_s_filt['ROSETTE_NUMBER']==i]
    sizes_s[i] = len(df_rosette)

## Random

In [53]:
list_rand = list(range(18))
tables_n = []

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_{i}_clustering.ran.fits"
    table = Table.read(url)
    tables_n.append(table)
    
rand_table_n = vstack(tables_n)

In [102]:
list_rand = list(range(18))
tables_s = []

for i in list_rand:
    url = f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_{i}_clustering.ran.fits"
    table = Table.read(url)
    tables_s.append(table)

rand_table_s = vstack(tables_s)

In [59]:
def cartesianas(data):
    data['distance'] = Planck18.comoving_distance(data['Z'])
    coords = SkyCoord(ra=data['RA'], dec=data['DEC'], distance=data['distance'], frame='icrs')
    data['x'] = coords.cartesian.x.value
    data['y'] = coords.cartesian.y.value
    data['z'] = coords.cartesian.z.value

    data['x'] = data['x'] * u.Mpc
    data['y'] = data['y'] * u.Mpc
    data['z'] = data['z'] * u.Mpc

In [104]:
def save_file(data, rand, sizes, n_random=100):

    rosettes = np.unique(data['ROSETTE_NUMBER'])

    for i in rosettes:
        rand_table_rosette = rand[rand['ROSETTE_NUMBER'] == i]

        for j in range(n_random):
            idx_random = np.random.choice(len(rand_table_rosette), size=sizes[i], replace=False)
            rand_subset = rand_table_rosette[idx_random]

            cartesianas(rand_subset)

            filename_rand = f"rand_rosette/LRG_rosette_{i}_random_{j}.ecsv"
            ascii.write(rand_subset, filename_rand, format='ecsv', overwrite=True)

In [87]:
save_file(data_n_filt,rand_table_n,sizes_n)

In [105]:
save_file(data_s_filt,rand_table_s,sizes_s)